In [3]:
from Bio.PDB import *
import pandas as pd
import warnings
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)
import csv
import numpy as np
from Bio.PDB.Polypeptide import three_to_one, one_to_three                           
import math
import requests as r
from Bio import SeqIO
from io import StringIO

In [104]:
def load_file(fName):
    df = pd.read_csv(fName)
    return df

In [105]:
def load_pd(fName):
    df = pd.read_csv(fName)
    df = df.dropna(subset = ['Affinity_mut_parsed', 'Affinity_wt_parsed', 'Temperature'])
    return df

skepiIn = "C:/Users/amber/Study_Documents/intern/inno/ppi with binding affinity/skempi/skempi_v2.csv"
ppi = load_pd(skepiIn)
print("Size of the data:", ppi.shape)


Size of the data: (6794, 29)


# wild-type library

## get dG data

In [106]:
def get_dg(ent_name, chain_ID0, chain_ID1, kd, dg_info):
    
    w_name0 = ent_name +'_'+chain_ID0
    w_name1 =  ent_name +'_'+chain_ID1
    #print(ent_name)
    # transfer kd unit to M
    
    dg = (8.314/4184)*(float(298)) * math.log(float(kd))
   
    info = w_name0 + " " +  w_name1 + " " + str(dg)
    dg_info.append(info)
    return dg_info

In [107]:
dg_info = []
wild_list = []

cnt_single_chain = cnt_both_chain = 0

f1 = open("./skempi/wild.seq.txt", "w")
f2 = open("./skempi/wild.dg.txt", "w")
#print(ppi.shape)

for i in range(6794):
    data = ppi.iloc[i]
    complex_id = data[0]
    if complex_id not in wild_list:
        ent_name = data[0][:4]
    #     .lower()
        #print(ent_name)
        chains = data[0].split('_')[1:]
        #print(chains)
        assert(len(chains) == 2)
    #     print(i, len(ppi))
        chain_ID0 = chains[0]
        chain_ID1 = chains[1]

        # there also exists cases the two chains in one side
        if len(chain_ID0) == 1 and len(chain_ID1) == 1:
            kd = data['Affinity_wt_parsed']
            #print(kd)
            temp = data['Temperature'].strip()[:3]
            #print(temp)
            
            get_dg(ent_name, chain_ID0, chain_ID1, kd, dg_info)
            wild_list.append(complex_id)

print(len(wild_list))

for elem in dg_info:
    print(elem, file=open("./skempi/wild.dg.txt", "a"))
  

225


there are 225 unique wild-type protein complex with two chains

## get sequencing data

### get uniprot id for complex in wild_list

This is the corresponding file, connects PDB code with uniprot code

In [108]:
filein = "C:/Users/amber/Study_Documents/intern/inno/ppi with binding affinity/pdb_chain_uniprot.tsv"
pdb_uniprot = pd.read_csv(filein, sep='\t')
pdb_uniprot.head

<bound method NDFrame.head of          PDB CHAIN SP_PRIMARY  RES_BEG  RES_END PDB_BEG PDB_END  SP_BEG  \
0       101m     A     P02185        1      154       0     153       1   
1       102l     A     P00720        1       40       1      40       1   
2       102l     A     P00720       42      165      41    None      41   
3       102m     A     P02185        1      154       0     153       1   
4       103l     A     P00720        1       40       1    None       1   
...      ...   ...        ...      ...      ...     ...     ...     ...   
616805  9xia     A     P24300        1      388       1    None       1   
616806  9xim     A     P12851        1      393    None     394       2   
616807  9xim     B     P12851        1      393    None     394       2   
616808  9xim     C     P12851        1      393    None     394       2   
616809  9xim     D     P12851        1      393    None     394       2   

        SP_END  
0          154  
1           40  
2          164  
3

In [109]:
list(pdb_uniprot.columns)

['PDB',
 'CHAIN',
 'SP_PRIMARY',
 'RES_BEG',
 'RES_END',
 'PDB_BEG',
 'PDB_END',
 'SP_BEG',
 'SP_END']

In [110]:
def get_seq_pdb (ent_name, chain_id, uniprot_dic, unstored_complex):
    
    data = pdb_uniprot.loc[(pdb_uniprot['PDB'] == ent_name.lower()) & (pdb_uniprot['CHAIN'] == chain_id), 'SP_PRIMARY']
    if data.empty:
        pdb_id = ent_name + "_" + chain_id
        unstored_complex.append(pdb_id)
    else:
        uniprot_id = data.values
        pdb_id = ent_name + "_" + chain_id
        #print(uniprot_id)
        #print(pdb_id)
        uniprot_dic[pdb_id] = uniprot_id[0]

unstored_complex = []
uniprot_dic = {}
for item in wild_list:
    #print(item)
    ent_name = item[:4]
    #     .lower()
    #print(ent_name)
    chains = item.split('_')[1:]
    #print(chains)
    assert(len(chains) == 2)
#     print(i, len(ppi))
    chain_ID0 = chains[0]
    chain_ID1 = chains[1]
    get_seq_pdb(ent_name, chain_ID0, uniprot_dic, unstored_complex)
    get_seq_pdb(ent_name, chain_ID1, uniprot_dic, unstored_complex)

#print(uniprot_dic)


In [111]:
print(len(uniprot_dic))
print(len(unstored_complex))

435
13


In [112]:
unstored_complex

['1JCK_A',
 '2NOJ_B',
 '4HRN_A',
 '4KRL_B',
 '4KRO_B',
 '4KRP_B',
 '3EQS_B',
 '3LNZ_B',
 '3EQY_C',
 '5M2O_B',
 '5XCO_B',
 '5UFE_B',
 '5UFQ_C']

There are 13 chains without uniprot code. Obtain this sequencing data from PDB lib

In [103]:
# def get_seq_pdb (ent_name, chain_id, uniprot_list):
    
#     data = pdb_uniprot.loc[(pdb_uniprot['PDB'] == ent_name.lower()) & (pdb_uniprot['CHAIN'] == chain_id), 'SP_PRIMARY']
#     #print(data)
#     uniprot_id = data.values
#     print(uniprot_id)
#     uniprot_list += list(uniprot_id)
    
# uniprot_list=[]
# get_seq_pdb('1CSE','E',uniprot_list)
# print(uniprot_list)

['B0FXJ2']
['B0FXJ2']


### get sequencing from uniprot id

In [113]:
uniprot_in = "C:/Users/amber/Study_Documents/intern/inno/ppi with binding affinity/uniprot.csv"
uniprot_seq = load_file(uniprot_in)
print(uniprot_seq.shape)


(566996, 2)


In [114]:
def seq_uniprot_lib (label):
    cID=label

    baseUrl="http://www.uniprot.org/uniprot/"
    currentUrl=baseUrl+cID+".fasta"
    response = r.post(currentUrl)
    cData=''.join(response.text)

    Seq=StringIO(cData)
    pSeq=list(SeqIO.parse(Seq,'fasta'))
    result =str(pSeq[0].seq)
    return result

#seq_uniprot_lib('Q07011')


In [115]:
def get_seq_uniprot (uniprot_id, pdb_id, seq_uniprot):
    
    data = uniprot_seq.loc[uniprot_seq['uniprot_id'] == uniprot_id,'pro_seq']
    if data.empty:
        seq_from_lib = seq_uniprot_lib(uniprot_id)
        seq_data = pdb_id + ' ' + seq_from_lib
        seq_uniprot.append(seq_data)
    else:
        seq = data.values[0]
        #print(seq)
        seq_data = pdb_id + ' ' + seq
        seq_uniprot.append(seq_data)
        #print(seq_uniprot)
    

In [116]:
seq_uniprot = []
for item in uniprot_dic:
    pdb_id = item
    #print(uniprot_id)
    uniprot_id = uniprot_dic[pdb_id]
    #print(pdb_id)
    get_seq_uniprot(uniprot_id, pdb_id, seq_uniprot)  

for elem in seq_uniprot:
    print(elem, file=open("./skempi/wild.seq.txt", "a"))

### get seq from PDB bank

download the pdb file

In [19]:
for item in unstored_complex:
    ent_name = item[:4]
    print(ent_name)
    pdbl = PDBList()
    assert(len(ent_name) ==4)
    pdbl.retrieve_pdb_file(ent_name, pdir = './skempi/PDBs/', file_format='pdb')

1JCK
2NOJ
4HRN
4KRL
4KRO
4KRP
3EQS
3LNZ
3EQY
5M2O
5XCO
5UFE
5UFQ


In [117]:
def get_seq(ent_name, chain, seqinfo):
#     print(ent_names, chain_ID0, chain_ID1)
    parser = PDBParser()
    structure = parser.get_structure(ent_name, './skempi/PDBs/pdb'+ ent_name.lower() + '.ent')
    #     assert(len(structure) == 1)
    model = structure[0]

    w_name = ent_name+'_'+ chain
    if w_name not in seqinfo:
        w_seq = ''

        chain = model[chain]
        ppb = PPBuilder()
        for pp in ppb.build_peptides(chain):
            w_seq += pp.get_sequence()
        #print(type(w_seq))
        sequence = w_name + " " + str(w_seq)
        seqinfo.append(sequence)
        
    return seqinfo

In [118]:
seqinfo = []

for item in unstored_complex:
    ent_name = item[:4]
    chain = item[-1]
    get_seq(ent_name, chain, seqinfo)
    

In [119]:
seqinfo

['1JCK_A AVTQSPRNKVAVTGGKVTLSCQQTNNHNNMYWYRQDTGHGLRLIHYSYGAGSTEKGDIPDGYKASRPSQEQFSLILELATPSQTSVYFCASGGGRGSYAEQFFGPGTRLTVLEDLRQVTPPKVSLFEPSKAEIANKQKATLVCLARGFFPDHVELSWWVNGKEVHSGVSTDPQAYKESNYSYCLSSRLRVSATFWHNPRNHFRCQVQFHGLSEEDKWPEGSPKPVTQNISAEAWGRAD',
 '2NOJ_B NKKVVDAQKAVELFKRTRTVATHRKAQRAVNLIHFQHSYEKKKLQRQIDLVLKYNTLK',
 '4HRN_A DLGKKLLEAARAGQDDEVRILMANGADVNAKDEYGLTPLYLATAHGHLEIVEVLLKNGADVNAVDAIGFTPLHLAAFIGHLEIAEVLLKHGADVNAQDKFGKTAFDISIGNGNEDLAEILQ',
 '4KRL_B QVKLEESGGGSVQTGGSLRLTCAASGRTSRSYGMGWFRQAPGKEREFVSGISWRGDSTGYADSVKGRFTISRDNAKNTVDLQMNSLKPEDTAIYYCAAAAGSAWYGTLYEYDYWGQGTQVTV',
 '4KRO_B QVQLQESLSCAASGRTFSSYAMGWFRQAPGKQREFVAAIRWSGGYTYYTDSVKGRFTISRDNAKTTVYLQMNSLKPEDTAVYYCAATYLSSDYSRYALPQRPLDYDYWGQGTQVTV',
 '4KRP_B EVQLVESGGGLLSCAASGRTFSSYAMGWFRQAPGKEREFVVAINWSSGSTYYADSVKGRFTISRDNAKNTMYLQMNSLKPEDTAVYYCAAGYQINSGNYNFKDYEYDYWGQGTQVT',
 '3EQS_B TSFAEYWNLLS',
 '3LNZ_B TSFAEYWALLS',
 '3EQY_C TSFAEYWNLLSP',
 '5M2O_B SVQKFPGDANCDGIVDISDAVLIMQTMANPSKYQMTDKGRINADVTGNSDGVTVLDAQFIQSYCLGLVELPPVE',
 

In [120]:
for elem in seqinfo:
    print(elem, file=open("./skempi/wild.seq.txt", "a"))

# mutation sequence

## dG data

In [121]:
mutated_dg_info= []
mutated_seq_list= []

f3 = open("./skempi/mut.seq.txt", "w")
f4 = open("./skempi/mut.dg.txt", "w")
#print(ppi.shape)

for i in range(6794):
    data = ppi.iloc[i]
    complex_id = data[0]
    ent_name = data[0][:4]
    chains = data[0].split('_')[1:]
    #print(chains)
    assert(len(chains) == 2)
    #     print(i, len(ppi))
    chain_ID0 = chains[0]
    chain_ID1 = chains[1]

    # there also exists cases the two chains in one side
    if len(chain_ID0) == 1 and len(chain_ID1) == 1:
        kd = data['Affinity_mut_parsed']
        #print(kd)
        temp = data['Temperature'].strip()[:3]
        #print(temp)
        mutation_site = data['Mutation(s)_cleaned']
        mutated_chain_ID0 = chain_ID0 + "_" + mutation_site
        mutated_chain_ID1 = chain_ID1 + "_" + mutation_site
        get_dg(ent_name, mutated_chain_ID0, mutated_chain_ID1, kd, mutated_dg_info)
        mutated_seq_list.append(ent_name + "_" + mutated_chain_ID0)
        mutated_seq_list.append(ent_name + "_" + mutated_chain_ID1)

for elem in mutated_dg_info:
    print(elem, file=open("./skempi/mut.dg.txt", "a"))

f3.close()
f4.close()

There is 4165 mutated complex pair with two chains

In [122]:
len(mutated_seq_list)
mutated_seq_list[40:50]

['1TM1_E_TI39D',
 '1TM1_I_TI39D',
 '1TM1_E_TI39A',
 '1TM1_I_TI39A',
 '1TM1_E_EI41A',
 '1TM1_I_EI41A',
 '1TM1_E_TI39A,EI41A',
 '1TM1_I_TI39A,EI41A',
 '1TM1_E_TI39D,EI41A',
 '1TM1_I_TI39D,EI41A']

## sequencing data

read wild-type txt into dic

In [123]:
seq_wild = {}
with open("./skempi/wild.seq.txt") as f:
    for line in f:
       (name, sequence) = line.split()
       seq_wild[name] = sequence


In [98]:
def mutation_on_seq(los):
    info = los.split("_")
    ent_name = info[0]
    chain_id = info[1]
    ent_id = "_".join(info[:2])
    #print(ent_id)
    lo_mutation = info[2].split(",")
    #print(lo_mutation)

    for item in lo_mutation:
        mutation_chain = item[1]
        mutation_pos = int(item[2:-1])
        print(mutation_pos)
        r_w = item[0]
        print(r_w)
        r_m = item[-1]
        
        if chain_id == mutation_chain:
            seq_w = seq_wild[ent_id]
            seq = list(seq_w)
            print(seq[mutation_pos - 1])
            if seq[mutation_pos -1] == r_w:
                seq[mutation_pos - 1] = r_m
                seq_m = "".join(seq)
                seq_mutated[los] = seq_m
            else:
                errormeg = "error in " + los
                print(errormeg)


mutation_on_seq('1TM1_I_TI39D,EI41A')

39
T
V
error in 1TM1_I_TI39D,EI41A
41
E
L
error in 1TM1_I_TI39D,EI41A


In [103]:
seq_wild['1TM1_I'][35:]

'AKKVILQDKPAAQIIVLPVGTIVTMEYRIDRVRLFVDRLDNIAQVPRVG'

**Then find the uniprot sequencing is not always corresponding to pdb sequencing**

have to use pdb file to extract sequencing data